In [ ]:
"""

Compute Bayesian weights based on Davison et al. 23 reference => dates 1997+

"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf
import os

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
outputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_raw = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'
inputpath_data=home_path+'/DATA/SUMMER_PAPER/interim/'


In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
BedMachine_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
file_BedMachine = uf.cut_domain_stereo(BedMachine_orig, map_lim, map_lim)
file_isf_conc = file_BedMachine['isf_conc']

grid_cell_area_file = xr.open_dataset(inputpath_data+'gridarea_ISMIP6_AIS_4000m_grid.nc').sel(x=file_isf.x,y=file_isf.y)
true_grid_cell_area = grid_cell_area_file['cell_area'].drop('lon').drop('lat')
cell_area_weight = true_grid_cell_area/(4000 * 4000)

lon = file_isf.longitude
lat = file_isf.latitude

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area_const = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area_const * cell_area_weight

isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')


In [ ]:
area_list = []
for kisf in tqdm(file_isf.Nisf):
    true_grid_cell_area_kisf = uf.choose_isf(grid_cell_area_weighted ,isf_stack_mask, kisf) 
    area_list.append(true_grid_cell_area_kisf.sum().assign_coords({'Nisf': kisf}))
area_BedMachine = xr.concat(area_list, dim = 'Nisf')/10**6

In [ ]:
area_BedMachine.plot(label='Bedmachine')
greene_file['area_abs'].mean('time').plot(label='Greene')
plt.legend()

In [ ]:
greene_file['area_abs'].mean('time').sel(Nisf=66)

In [ ]:
area_BedMachine.sel(Nisf=66)

In [ ]:
plt.figure()
plt.scatter(np.arange(len(area_BedMachine.Nisf)),area_BedMachine, label='area BedMachine (us)')
plt.scatter(np.arange(len(area_BedMachine.Nisf)),greene_file['area_abs'].mean('time'), label='area Greene averaged over time')
plt.ylim(0,1e5)
plt.xticks(ticks=np.arange(len(file_isf['isf_name'])),labels=file_isf['isf_name'].values, rotation=90)
plt.legend()

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

REFERENCE

In [ ]:
inputpath_davison = home_path+'/DATA/SUMMER_PAPER/raw/'
davison_file0 = xr.open_dataset(inputpath_davison + 'steadystate_davison23.nc')
davison_file = xr.open_dataset(inputpath_davison + 'varying_conditions_davison23.nc')
greene_file = xr.open_dataset(inputpath_davison + 'area_isf_greene22.nc')

In [ ]:
## this would be correcting the obs but we want to correct the models
# davison_file['melt_abs_corr'] = davison_file['melt_abs'] /  greene_file['area_abs'] * area_BedMachine
# davison_file['SMB_abs_corr'] = davison_file['SMB_abs'] /  greene_file['area_abs'] * area_BedMachine

In [ ]:
mass_balance_davison_steady = davison_file0['basal_melt_obs'] - davison_file0['SMB_obs'] + davison_file0['calving_obs'] - davison_file['discharge_abs']
mass_balance_davison_steady_unc = np.sqrt(davison_file0['basal_melt_unc']**2 + davison_file0['SMB_unc']**2 + davison_file0['calving_obs']**2 + davison_file0['discharge_unc']**2)

In [ ]:
mass_balance_davison_varying = davison_file['melt_abs'] - davison_file['SMB_abs'] + davison_file['calving_abs'] - davison_file['discharge_abs']
mass_balance_davison_varying_unc = np.sqrt(davison_file['melt_unc']**2 + davison_file['SMB_unc']**2 + davison_file['calving_unc']**2 + davison_file0['discharge_unc']**2)

SMB

In [ ]:
inputpath_atmo_Chris = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_atmo_Nico = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/SMB_EMULATED/'
scenario = 'historical'

melt_atmo_list = []
for mod in ['CESM2','CNRM-CM6-1','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']:
    if os.path.exists(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc'):
        melt_atmo = xr.open_dataset(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
        melt_atmo['time'] = melt_atmo['time'].dt.year
        melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
    else:
        melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
        melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
        
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2-WACCM','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 
           'MRI-ESM2-0']:
    melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
    melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
    
melt_atmo_all = xr.concat(melt_atmo_list, dim='model')

In [ ]:
melt_atmo_all_corr = melt_atmo_all / area_BedMachine * greene_file['area_abs']

BASAL MELT

In [ ]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNsmall']

In [ ]:
## Melt outputpath
Gt_allmod_list = []
box1_allmod_list = []

scenario = 'historical'
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']: #
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list = []
    for mparam in param_classic_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoISMIP.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y'].sel(time=range(1980,2015))

    melt1D_list = []
    for mparam in param_NN_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoISMIP.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')

    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    
    Gt_allmod_list.append(Gt_all.assign_coords({'model': mod}))

Gt_allmod = xr.concat(Gt_allmod_list, dim='model').sel(time=range(1980,2015))
    

In [ ]:
Gt_allmod_corr = Gt_allmod / area_BedMachine * greene_file['area_abs']

In [ ]:
## Melt outputpath
Gt_allmod_list = []
box1_allmod_list = []

scenario = 'historical'
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']: #
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list = []
    for mparam in param_classic_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_notISMIP.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y'].sel(time=range(1980,2015))

    melt1D_list = []
    for mparam in param_NN_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_notISMIP.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')

    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    
    Gt_allmod_list.append(Gt_all.assign_coords({'model': mod}))

Gt_allmod_notISMIP = xr.concat(Gt_allmod_list, dim='model').sel(time=range(1980,2015))
    

In [ ]:
## Melt outputpath
Gt_allmod_list = []
box1_allmod_list = []

scenario = 'historical'
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']: #
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list = []
    for mparam in param_classic_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoNEMO.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y'].sel(time=range(1980,2015))

    melt1D_list = []
    for mparam in param_NN_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoNEMO.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')

    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    
    Gt_allmod_list.append(Gt_all.assign_coords({'model': mod}))

Gt_allmod_anoNEMO = xr.concat(Gt_allmod_list, dim='model').sel(time=range(1980,2015))
    

In [ ]:
Gt_allmod_notISMIP_corr = Gt_allmod_notISMIP / area_BedMachine * greene_file['area_abs']
Gt_allmod_anoNEMO_corr = Gt_allmod_anoNEMO / area_BedMachine * greene_file['area_abs']

In [ ]:
# MERGE, CHOOSING THE LOWEST DIFFERENCE FOR EACH ICE SHELF USING BASAL MELT
diff_mod_obs_bmb_anoISMIP = Gt_allmod_corr.sel(time=slice(1997,2014)).mean('time') - davison_file['melt_abs'].sel(time=slice(1997,2014)).mean('time')
diff_mod_obs_bmb_anoNEMO = Gt_allmod_anoNEMO_corr.sel(time=slice(1997,2014)).mean('time') - davison_file['melt_abs'].sel(time=slice(1997,2014)).mean('time')

#sigma_obs_bmb = np.sqrt((davison_file['melt_unc']**2).max('time')) 
#sigma_mod_bmb = sigma_obs_bmb * 10

#s_j_bmb_anoNEMO = np.exp(-((diff_mod_obs_bmb_anoNEMO)**2/(sigma_obs_bmb**2 + sigma_mod_bmb**2)))
#s_j_bmb_anoISMIP = np.exp(-((diff_mod_obs_bmb_anoISMIP)**2/(sigma_obs_bmb**2 + sigma_mod_bmb**2)))

ano_choice_list = []

for kisf in file_isf.Nisf:
    #mean_weights_anoNEMO = s_j_bmb_anoNEMO.sel(Nisf=kisf).mean()
    #mean_weights_anoISMIP = s_j_bmb_anoISMIP.sel(Nisf=kisf).mean()
    mean_diff_anoNEMO = diff_mod_obs_bmb_anoNEMO.sel(Nisf=kisf).median()
    mean_diff_anoISMIP = diff_mod_obs_bmb_anoISMIP.sel(Nisf=kisf).median()
    
    #if mean_weights_anoNEMO > mean_weights_anoISMIP:
    if mean_diff_anoNEMO < mean_diff_anoISMIP:
        ano_choice_list.append('NEMO')
    else:
        ano_choice_list.append('ISMIP')

#for kisf in file_isf.Nisf:
#    largediff_anoISMIP = (diff_mod_obs_bmb_anoISMIP.sel(Nisf=kisf) > sigma_obs_bmb.sel(Nisf=kisf)).sum(['param','model'])
#    largediff_anoNEMO = (diff_mod_obs_bmb_anoNEMO.sel(Nisf=kisf) > sigma_obs_bmb.sel(Nisf=kisf)).sum(['param','model'])
#    
#    if largediff_anoISMIP > largediff_anoNEMO:
#        ano_choice_list.append('NEMO')
#    else:
#        ano_choice_list.append('ISMIP')

ano_choice = xr.DataArray(data=np.array(ano_choice_list), dims=['Nisf']).assign_coords({'Nisf': file_isf.Nisf})

Check the BMB and the ano choice

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = davison_file['melt_abs'].sel(time=slice(1997,2014)).mean('time').sel(Nisf=kisf).values
    melt_obs_unc_kisf = sigma_obs_bmb.sel(Nisf=kisf).values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')

    for mmod in diff_mod_obs_bmb_anoISMIP.model: #.drop_sel(model='GISS-E2-1-H')
        for mparam in diff_mod_obs_bmb_anoISMIP.param:
    
            ax[i].plot(Gt_allmod_corr.time, Gt_allmod_corr.sel(param=mparam,model=mmod,Nisf=kisf), color='deepskyblue',alpha=0.06)
            ax[i].plot(Gt_allmod_corr.time, Gt_allmod_anoNEMO_corr.sel(param=mparam,model=mmod,Nisf=kisf), color='blue',alpha=0.06)
            ax[i].axhline(Gt_allmod_corr.sel(param=mparam,model=mmod,Nisf=kisf).mean('time'), color='deepskyblue')
            ax[i].axhline(Gt_allmod_anoNEMO_corr.sel(param=mparam,model=mmod,Nisf=kisf).mean('time'), color='blue')
    
    #ax[i].set_title(ano_choice.sel(Nisf=kisf))

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values)+' '+str(ano_choice.sel(Nisf=kisf).values))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'compare_bmb_Davison_varying_anoISMIPandanoNEMO.pdf')

GL FLUX FOR ABUMIP

In [ ]:
#GL_flux_ABUMIP = xr.open_dataset(outputpath_GL + 'all_GL_fluxes.nc')
GL_flux = xr.open_dataset(outputpath_GL + 'all_GL_fluxes_varying_m.nc')

In [ ]:
GL_flux_ag = xr.Dataset()
GL_flux_ag['flux_Gt_ref'] = xr.concat([GL_flux['flux_Gt_ref_m1'].assign_coords({'m': 1}),
                                       GL_flux['flux_Gt_ref_m3'].assign_coords({'m': 3}),
                                       GL_flux['flux_Gt_ref_m5'].assign_coords({'m': 5})], dim='m')
GL_flux_ag['flux_Gt_ABUMIP'] = xr.concat([GL_flux['flux_Gt_ABUMIP_m1'].assign_coords({'m': 1}),
                                       GL_flux['flux_Gt_ABUMIP_m3'].assign_coords({'m': 3}),
                                       GL_flux['flux_Gt_ABUMIP_m5'].assign_coords({'m': 5})], dim='m')

SUM OF ALL TERMS

In [ ]:
mass_balance_simu_anoISMIP = Gt_allmod_corr - melt_atmo_all_corr + davison_file0['calving_obs'] - GL_flux_ag['flux_Gt_ref']
mass_balance_simu_notISMIP = Gt_allmod_notISMIP_corr - melt_atmo_all_corr + davison_file0['calving_obs'] - GL_flux_ag['flux_Gt_ref']
mass_balance_simu_anoNEMO = Gt_allmod_anoNEMO_corr - melt_atmo_all_corr + davison_file0['calving_obs'] - GL_flux_ag['flux_Gt_ref']

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison_varying.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    melt_obs_unc_kisf = mass_balance_davison_varying_unc.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')

    for mmod in mass_balance_simu_notISMIP.model: #.drop_sel(model='GISS-E2-1-H')
        for mparam in mass_balance_simu_notISMIP.param:
            for mchoice in mass_balance_simu_notISMIP.m:
    
                ax[i].plot(mass_balance_simu_notISMIP.time, mass_balance_simu_anoISMIP.sel(param=mparam,model=mmod,m=mchoice,Nisf=kisf), color='deepskyblue',alpha=0.06)
                ax[i].plot(mass_balance_simu_notISMIP.time, mass_balance_simu_anoNEMO.sel(param=mparam,model=mmod,m=mchoice,Nisf=kisf), color='blue',alpha=0.06)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'compare_massbalanceinclGLflux_Davison_varying_anoISMIPandanoNEMO.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison_varying.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    melt_obs_unc_kisf = mass_balance_davison_varying_unc.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')

    for mmod in mass_balance_simu_notISMIP.model.drop_sel(model='GISS-E2-1-H'):
        for mparam in mass_balance_simu_notISMIP.param:
    
            ax[i].plot(mass_balance_simu_notISMIP.time, mass_balance_simu_notISMIP.sel(param=mparam,model=mmod,Nisf=kisf), color='blue',alpha=0.06)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'compare_massbalance_Davison_varying_notISMIP.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison_varying.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    melt_obs_unc_kisf = mass_balance_davison_varying_unc.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')

    for mmod in mass_balance_simu_anoNEMO.model.drop_sel(model='GISS-E2-1-H'):
        for mparam in mass_balance_simu_anoNEMO.param:
    
            ax[i].plot(mass_balance_simu_notISMIP.time, mass_balance_simu_anoNEMO.sel(param=mparam,model=mmod,Nisf=kisf), color='blue',alpha=0.06)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'compare_massbalance_Davison_varying_anoNEMO.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison_steady.sel(Nisf=kisf).values
    melt_obs_unc_kisf = mass_balance_davison_unc.sel(Nisf=kisf).values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1980,2009), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')

    for mmod in mass_balance_simu.model:
        for mparam in mass_balance_simu.param:
    
            ax[i].plot(np.arange(1980,2009), mass_balance_simu.sel(param=mparam,model=mmod,time=range(1980,2009),Nisf=kisf), color='blue',alpha=0.06)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'compare_massbalance_Davison_steady.pdf')

In [ ]:
diff_mod_obs_anoISMIP = mass_balance_simu_anoISMIP.mean('time') - mass_balance_davison_varying.mean('time')
diff_mod_obs_notISMIP = mass_balance_simu_notISMIP.mean('time') - mass_balance_davison_varying.mean('time')
diff_mod_obs_anoNEMO = mass_balance_simu_anoNEMO.mean('time') - mass_balance_davison_varying.mean('time')


sigma_obs = mass_balance_davison_varying_unc.max('time')
sigma_mod = sigma_obs * 5

In [ ]:
s_j = np.exp(-((diff_mod_obs_anoISMIP)**2/(sigma_obs**2 + sigma_mod**2)))

weight = (s_j / (s_j.sum(['model','param','m'])))

weight_clean_anoISMIP = weight#.where(np.isfinite(weight),0)
weight_clean_anoISMIP.to_dataset(name='bay_weights').to_netcdf(outputpath_weights + 'bayesian_weights_davison_varying_anoISMIP.nc')

In [ ]:
s_j = (np.exp(-((diff_mod_obs_anoISMIP)**2/(sigma_obs**2 + sigma_mod**2)))).sel(model=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                       'CESM2-WACCM','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                       'MRI-ESM2-0','UKESM1-0-LL'])

weight = (s_j / (s_j.sum(['model','param','m'])))

weight_clean_anoISMIP = weight#.where(np.isfinite(weight),0)
weight_clean_anoISMIP.to_dataset(name='bay_weights').to_netcdf(outputpath_weights + 'bayesian_weights_davison_varying_anoISMIP_2300.nc')

In [ ]:
s_j = np.exp(-((diff_mod_obs_notISMIP)**2/(sigma_obs**2 + sigma_mod**2)))

weight = (s_j / (s_j.sum(['model','param','m'])))

weight_clean_notISMIP = weight#.where(np.isfinite(weight),0)
weight_clean_notISMIP.to_dataset(name='bay_weights').to_netcdf(outputpath_weights + 'bayesian_weights_davison_varying_notISMIP.nc')

In [ ]:
s_j = np.exp(-((diff_mod_obs_anoNEMO)**2/(sigma_obs**2 + sigma_mod**2)))

weight = (s_j / (s_j.sum(['model','param','m'])))

weight_clean_anoNEMO = weight#.where(np.isfinite(weight),0)
weight_clean_anoNEMO.to_dataset(name='bay_weights').to_netcdf(outputpath_weights + 'bayesian_weights_davison_varying_anoNEMO.nc')

In [ ]:
s_j = np.exp(-((diff_mod_obs_anoNEMO)**2/(sigma_obs**2 + sigma_mod**2))).sel(model=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                       'CESM2-WACCM','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                       'MRI-ESM2-0','UKESM1-0-LL'])

weight = (s_j / (s_j.sum(['model','param','m'])))

weight_clean_anoNEMO = weight#.where(np.isfinite(weight),0)
weight_clean_anoNEMO.to_dataset(name='bay_weights').to_netcdf(outputpath_weights + 'bayesian_weights_davison_varying_anoNEMO_2300.nc')

In [ ]:
ensmean_weighted_anoISMIP = uf.weighted_mean(mass_balance_simu_anoISMIP, ['model','param','m'], weight_clean_anoISMIP)
ensmean_nonweighted_anoISMIP = mass_balance_simu_anoISMIP.mean(['model','param','m'])

da_weighted_anoISMIP = mass_balance_simu_anoISMIP.weighted(weight_clean_anoISMIP)

ensstat_nonweighted_anoISMIP = mass_balance_simu_anoISMIP.quantile([0.05,0.10,0.33,0.5,0.66,0.90,0.95],dim=['model','param','m'])
ensstat_weighted_anoISMIP = da_weighted_anoISMIP.quantile([0.05,0.10,0.33,0.5,0.66,0.90,0.95],dim=['model','param','m'])

In [ ]:
ensmean_weighted_notISMIP = uf.weighted_mean(mass_balance_simu_notISMIP, ['model','param','m'], weight_clean_notISMIP)
ensmean_nonweighted_notISMIP = mass_balance_simu_notISMIP.mean(['model','param','m'])

da_weighted_notISMIP = mass_balance_simu_notISMIP.weighted(weight_clean_notISMIP)

ensstat_nonweighted_notISMIP = mass_balance_simu_notISMIP.quantile([0.05,0.10,0.33,0.5,0.66,0.90,0.95],dim=['model','param','m'])
ensstat_weighted_notISMIP = da_weighted_notISMIP.quantile([0.05,0.10,0.33,0.5,0.66,0.90,0.95],dim=['model','param','m'])

In [ ]:
ensmean_weighted_anoNEMO = uf.weighted_mean(mass_balance_simu_anoNEMO, ['model','param','m'], weight_clean_anoNEMO)
ensmean_nonweighted_anoNEMO = mass_balance_simu_anoNEMO.mean(['model','param','m'])

da_weighted_anoNEMO = mass_balance_simu_anoNEMO.weighted(weight_clean_anoNEMO)

ensstat_nonweighted_anoNEMO = mass_balance_simu_anoNEMO.quantile([0.05,0.10,0.33,0.5,0.66,0.90,0.95],dim=['model','param','m'])
ensstat_weighted_anoNEMO = da_weighted_anoNEMO.quantile([0.05,0.10,0.33,0.5,0.66,0.90,0.95],dim=['model','param','m'])

In [ ]:
mass_balance_simu_combined_list = []
bay_weight_list = []

for kisf in file_isf.Nisf:
    if ano_choice.sel(Nisf=kisf).values == 'NEMO':
        mass_balance_simu_combined_list.append(mass_balance_simu_anoNEMO.sel(Nisf=kisf))
        bay_weight_list.append(weight_clean_anoNEMO.sel(Nisf=kisf))
        
    elif ano_choice.sel(Nisf=kisf).values == 'ISMIP':
        mass_balance_simu_combined_list.append(mass_balance_simu_anoISMIP.sel(Nisf=kisf))
        bay_weight_list.append(weight_clean_anoISMIP.sel(Nisf=kisf))

mass_balance_simu_combined = xr.concat(mass_balance_simu_combined_list, dim='Nisf')
bay_weight_combined = xr.concat(bay_weight_list, dim='Nisf')

xr.merge([ano_choice.rename('ano_choice'),bay_weight_combined.rename('bay_weights')]).to_netcdf(outputpath_weights + 'NEMOorISMIP_choice_and_weights.nc')

In [ ]:
mass_balance_simu_combined_list = []
bay_weight_list = []

for kisf in file_isf.Nisf:
    if ano_choice.sel(Nisf=kisf).values == 'NEMO':
        mass_balance_simu_combined_list.append(mass_balance_simu_anoNEMO.sel(Nisf=kisf))
        bay_weight_list.append(weight_clean_anoNEMO.sel(Nisf=kisf))
        
    elif ano_choice.sel(Nisf=kisf).values == 'ISMIP':
        mass_balance_simu_combined_list.append(mass_balance_simu_anoISMIP.sel(Nisf=kisf))
        bay_weight_list.append(weight_clean_anoISMIP.sel(Nisf=kisf))

mass_balance_simu_combined = xr.concat(mass_balance_simu_combined_list, dim='Nisf')
bay_weight_combined = xr.concat(bay_weight_list, dim='Nisf').sel(model=['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                       'CESM2-WACCM','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                       'MRI-ESM2-0','UKESM1-0-LL'])

xr.merge([ano_choice.rename('ano_choice'),bay_weight_combined.rename('bay_weights')]).to_netcdf(outputpath_weights + 'NEMOorISMIP_choice_and_weights_2300.nc')

In [ ]:
ensmean_weighted_combined = uf.weighted_mean(mass_balance_simu_combined, ['model','param','m'], bay_weight_combined)
ensmean_nonweighted_combined = mass_balance_simu_combined.mean(['model','param','m'])

da_weighted_combined = mass_balance_simu_combined.weighted(bay_weight_combined)

ensstat_nonweighted_combined = mass_balance_simu_combined.quantile([0.05,0.10,0.33,0.5,0.66,0.90,0.95],dim=['model','param','m'])
ensstat_weighted_combined = da_weighted_combined.quantile([0.05,0.10,0.33,0.5,0.66,0.90,0.95],dim=['model','param','m'])

In [ ]:
### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0

sigma_tot = np.sqrt(sigma_obs**2 + sigma_mod**2)

for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison_varying.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    melt_obs_unc_kisf = mass_balance_davison_varying_unc.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-sigma_tot.sel(Nisf=kisf), y2=melt_obs_kisf+sigma_tot.sel(Nisf=kisf), color='lightgrey',alpha=0.2)

    
    ax[i].plot(ensstat_weighted_combined.time, ensstat_weighted_combined.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].plot(ensstat_weighted_combined.time, ensstat_weighted_combined.sel(quantile=0.05,Nisf=kisf), color='deepskyblue',linestyle='--')
    ax[i].plot(ensstat_weighted_combined.time, ensstat_weighted_combined.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',linestyle='--')

    #ax[i].plot(ensstat_weighted.time, ensstat_nonweighted.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].fill_between(x=ensstat_weighted_combined.time, y1=ensstat_weighted_combined.sel(quantile=0.33,Nisf=kisf), y2=ensstat_weighted_combined.sel(quantile=0.66,Nisf=kisf), color='deepskyblue',alpha=0.2)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'ensmean_weighted_allmod_allmassbalance_allm_combined.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison_varying.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    melt_obs_unc_kisf = mass_balance_davison_varying_unc.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')
    
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoISMIP.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoISMIP.sel(quantile=0.05,Nisf=kisf), color='deepskyblue',linestyle='--')
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoISMIP.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',linestyle='--')

    #ax[i].plot(ensstat_weighted.time, ensstat_nonweighted.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].fill_between(x=ensstat_weighted_anoISMIP.time, y1=ensstat_weighted_anoISMIP.sel(quantile=0.33,Nisf=kisf), y2=ensstat_weighted_anoISMIP.sel(quantile=0.66,Nisf=kisf), color='deepskyblue',alpha=0.2)
    #ax[i].fill_between(x=ensstat_weighted.time, y1=ensstat_nonweighted.sel(quantile=0.05,Nisf=kisf), y2=ensstat_nonweighted.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',alpha=0.2)
    ax[i].plot(ensstat_weighted_notISMIP.time, ensstat_weighted_notISMIP.sel(quantile=0.5,Nisf=kisf), color='orange')
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_notISMIP.sel(quantile=0.05,Nisf=kisf), color='orange',linestyle='--')
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_notISMIP.sel(quantile=0.95,Nisf=kisf), color='orange',linestyle='--')
    #ax[i].plot(ensstat_weighted.time, ensstat_nonweighted.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].fill_between(x=ensstat_weighted_notISMIP.time, y1=ensstat_weighted_notISMIP.sel(quantile=0.33,Nisf=kisf), y2=ensstat_weighted_notISMIP.sel(quantile=0.66,Nisf=kisf), color='orange',alpha=0.2)
    #ax[i].fill_between(x=ensstat_weighted.time, y1=ensstat_nonweighted.sel(quantile=0.05,Nisf=kisf), y2=ensstat_nonweighted.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',alpha=0.2)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'ensmean_weighted_allmod_allmassbalance_ano_and_notISMIP.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison_varying.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    melt_obs_unc_kisf = mass_balance_davison_varying_unc.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')
    
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoISMIP.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoISMIP.sel(quantile=0.05,Nisf=kisf), color='deepskyblue',linestyle='--')
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoISMIP.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',linestyle='--')

    #ax[i].plot(ensstat_weighted.time, ensstat_nonweighted.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].fill_between(x=ensstat_weighted_anoISMIP.time, y1=ensstat_weighted_anoISMIP.sel(quantile=0.33,Nisf=kisf), y2=ensstat_weighted_anoISMIP.sel(quantile=0.66,Nisf=kisf), color='deepskyblue',alpha=0.2)
    #ax[i].fill_between(x=ensstat_weighted.time, y1=ensstat_nonweighted.sel(quantile=0.05,Nisf=kisf), y2=ensstat_nonweighted.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',alpha=0.2)
    ax[i].plot(ensstat_weighted_anoNEMO.time, ensstat_weighted_anoNEMO.sel(quantile=0.5,Nisf=kisf), color='orange')
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoNEMO.sel(quantile=0.05,Nisf=kisf), color='orange',linestyle='--')
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoNEMO.sel(quantile=0.95,Nisf=kisf), color='orange',linestyle='--')
    #ax[i].plot(ensstat_weighted.time, ensstat_nonweighted.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].fill_between(x=ensstat_weighted_anoNEMO.time, y1=ensstat_weighted_anoNEMO.sel(quantile=0.33,Nisf=kisf), y2=ensstat_weighted_anoNEMO.sel(quantile=0.66,Nisf=kisf), color='orange',alpha=0.2)
    #ax[i].fill_between(x=ensstat_weighted.time, y1=ensstat_nonweighted.sel(quantile=0.05,Nisf=kisf), y2=ensstat_nonweighted.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',alpha=0.2)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'ensmean_weighted_allmod_allmassbalance_ano_and_anoNEMO.pdf')

In [ ]:
melt_Paolo = xr.open_dataset(inputpath_raw + 'basal_melt_Gt_Paolo23.nc')

In [ ]:
melt_Paolo

In [ ]:
mass_balance_davison_steady_paolo = melt_Paolo['melt_Gt'].mean('time') - davison_file0['SMB_obs'] + davison_file0['calving_obs']
mass_balance_davison_steady_paolo_unc = np.sqrt(melt_Paolo['melt_Gt_unc'].max('time')**2 + davison_file0['SMB_unc']**2 + davison_file0['calving_obs']**2)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison_varying.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    melt_obs_unc_kisf = mass_balance_davison_varying_unc.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    
    melt_obs2_kisf = mass_balance_davison_steady_paolo.sel(Nisf=kisf).values
    melt_obs2_unc_kisf = mass_balance_davison_steady_paolo_unc.sel(Nisf=kisf).values
    
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')
    
    ax[i].axhline(y=melt_obs2_kisf,color='green')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs2_kisf-melt_obs2_unc_kisf, y2=melt_obs2_kisf+melt_obs2_unc_kisf, color='green', alpha=0.2)
    
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoISMIP.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoISMIP.sel(quantile=0.05,Nisf=kisf), color='deepskyblue',linestyle='--')
    ax[i].plot(ensstat_weighted_anoISMIP.time, ensstat_weighted_anoISMIP.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',linestyle='--')
    ax[i].fill_between(x=ensstat_weighted_anoISMIP.time, y1=ensstat_weighted_anoISMIP.sel(quantile=0.33,Nisf=kisf), y2=ensstat_weighted_anoISMIP.sel(quantile=0.66,Nisf=kisf), color='deepskyblue',alpha=0.2)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'ensmean_weighted_allmod_allmassbalance_anoISMIP_withPaolo.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison_varying.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    melt_obs_unc_kisf = mass_balance_davison_varying_unc.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time').values
    
    melt_obs2_kisf = mass_balance_davison_steady_paolo.sel(Nisf=kisf).values
    melt_obs2_unc_kisf = mass_balance_davison_steady_paolo_unc.sel(Nisf=kisf).values
    
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')
    
    ax[i].axhline(y=melt_obs2_kisf,color='green')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs2_kisf-melt_obs2_unc_kisf, y2=melt_obs2_kisf+melt_obs2_unc_kisf, color='green', alpha=0.2)

    for mmod in mass_balance_simu.model.drop_sel(model='GISS-E2-1-H'):
        for mparam in mass_balance_simu.param:
    
            ax[i].plot(mass_balance_simu_notISMIP.time, mass_balance_simu_anoISMIP.sel(param=mparam,model=mmod,Nisf=kisf), color='blue',alpha=0.06)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'compare_massbalance_Davison_varying_anoISMIP_withPaolo.pdf')

In [ ]:

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison.sel(Nisf=kisf).values
    melt_obs_unc_kisf = mass_balance_davison_unc.sel(Nisf=kisf).values
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=ensstat_weighted.time, y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')
    
    ax[i].plot(ensstat_weighted.time, ensstat_weighted.sel(quantile=0.5,Nisf=kisf), color='blue')
    #ax[i].plot(ensstat_weighted.time, ensstat_nonweighted.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    ax[i].fill_between(x=ensstat_weighted.time, y1=ensstat_weighted.sel(quantile=0.05,Nisf=kisf), y2=ensstat_weighted.sel(quantile=0.95,Nisf=kisf), color='blue',alpha=0.2)
    #ax[i].fill_between(x=ensstat_weighted.time, y1=ensstat_nonweighted.sel(quantile=0.05,Nisf=kisf), y2=ensstat_nonweighted.sel(quantile=0.95,Nisf=kisf), color='deepskyblue',alpha=0.2)
    
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ref'].sel(Nisf=kisf), color='orange')
    ax[i].axhline(y=GL_flux_ABUMIP['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='red')


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'ensmean_weighted_allmod_allmassbalance_withABUMIPlimit_anoISMIP.pdf')